## Import Needed Modules

In [1]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
# from torchvision.models import resnet18, VGG16_Weights, ResNet18_Weights, AlexNet_Weights
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report
)
import pandas as pd
import logging
from datetime import datetime
import math
from typing import Optional, Tuple, List, Dict, Any
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
import io
import seaborn as sns
import logging
from io import BytesIO
from PIL import Image
import csv
import pandas as pd

2025-05-05 15:27:04.509655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746458824.744974      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746458824.812368      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Get Data

In [4]:
# Baseline: RGB
train_data_dataloader_baseline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
    ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_data_dataloader_baseline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom: RGB
train_data_dataloader = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    #transforms.RandomApply([
        #transforms.RandomRotation(degrees=15),
        #transforms.ColorJitter(brightness=0.2, contrast=0.2),
        #transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
    #], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_data_dataloader = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom: Gray
train_data_dataloader_gry = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    #transforms.RandomApply([
        #transforms.RandomRotation(degrees=15),
        #transforms.ColorJitter(brightness=0.2, contrast=0.2),
        #transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
    #], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_data_dataloader_gry = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# dataset: https://drive.google.com/drive/folders/1d1ArqNswahfsmP6h-fm3WCvL47dVOpxA?usp=drive_link

# Define the base directory for data
base_data_dir = '/input/brain-tumor-mri-dataset'

# 1. Baseline: RGB
# train_dataset_base = ImageFolder(f'{base_data_dir}/Training', transform=train_data_dataloader_baseline)
# train_loader_base = DataLoader(train_dataset_base, batch_size=128, shuffle=True)  # batch_size set to 128 due to "CUDA out of memory" when using VGG16 model

# val_dataset_base = ImageFolder(f'{base_data_dir}/Testing', transform=val_data_dataloader_baseline)
# val_loader_base = DataLoader(val_dataset_base, batch_size=128, shuffle=False)

# 2. Custom: RGB
train_dataset = ImageFolder(f'{base_data_dir}/Training', transform=train_data_dataloader)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = ImageFolder(f'{base_data_dir}/Testing', transform=val_data_dataloader)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# 3. Custom: Gray
train_dataset_gry = ImageFolder(f'{base_data_dir}/Training', transform=train_data_dataloader_gry)
train_loader_gry = DataLoader(train_dataset_gry, batch_size=128, shuffle=True)

val_dataset_gry = ImageFolder(f'{base_data_dir}/Testing', transform=val_data_dataloader_gry)
val_loader_gry = DataLoader(val_dataset_gry, batch_size=128, shuffle=False)


## Custom Model Architecture

In [5]:
class EfficientMultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        # Use a single projection for QKV to reduce computation
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        B, L, C = x.shape

        # Efficient QKV projection
        qkv = self.qkv_proj(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: t.view(B, L, self.num_heads, self.head_dim).transpose(1, 2), qkv)

        # Use einsum for more efficient attention computation
        attn_weights = torch.einsum('bhqd,bhkd->bhqk', q, k) * self.scale

        if mask is not None:
            attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))

        attn_probs = F.softmax(attn_weights, dim=-1)
        attn_probs = self.dropout(attn_probs)

        context = torch.einsum('bhqk,bhkd->bhqd', attn_probs, v)
        context = context.transpose(1, 2).contiguous().view(B, L, C)

        return self.out_proj(context)

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.05):
        super().__init__()
        self.attention = EfficientMultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            #nn.Dropout(dropout),
        )
    def forward(self, x):
        attn_out = self.attention(self.norm1(x))
        x = x + attn_out
        mlp_out = self.mlp(self.norm2(x))
        x = x + mlp_out
        return x

    # def forward(self, x):
    #     x = x + self.attention(x)
    #     x = x + self.mlp(x)
    #     return x

class RotaryPositionalEmbedding(nn.Module):
    def __init__(self, dim, base=10000):
        super().__init__()
        inv_freq = 1. / (base ** (torch.arange(0., dim, 2.) / dim))
        self.register_buffer('inv_freq', inv_freq)

    def forward(self, x, seq_dim=1):
        t = torch.arange(x.shape[seq_dim], device=x.device).type_as(self.inv_freq)
        sinusoid_inp = torch.einsum("i,j->ij", t, self.inv_freq)
        emb = torch.cat((sinusoid_inp.sin(), sinusoid_inp.cos()), dim=-1)
        return emb[None, :, :]

class HybridFPNTransformer_gry(nn.Module):
    def __init__(self, num_classes, image_size=224, patch_size=16,
                 embed_dim=128, num_heads=8, transformer_depth=4):
        super().__init__()
        
        # Stem network remains the same - processes raw images
        self.stem = nn.Sequential(
            nn.Conv2d(1, embed_dim*2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim*2),
            nn.ReLU(inplace=True),
            nn.Conv2d(embed_dim*2, embed_dim, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(inplace=True)
        )

        # Patch embedding remains the same
        self.patch_embed = nn.Conv2d(
            embed_dim, embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )

        # Rotary positional embeddings for spatial awareness
        self.rotary_emb = RotaryPositionalEmbedding(embed_dim)

        # Transformer blocks remain unchanged
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, embed_dim)
            for _ in range(transformer_depth)
        ])

        # Simplified classification head - directly from transformer output
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(embed_dim, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # Initial feature extraction
        x = self.stem(x)
        x_patches = self.patch_embed(x)

        # Reshape for transformer processing
        B, C, H, W = x_patches.shape
        x_patches = x_patches.flatten(2).transpose(1, 2)

        # Add positional information
        rotary_pos_emb = self.rotary_emb(x_patches)
        x_patches = x_patches + rotary_pos_emb

        # Process through transformer blocks
        for transformer_block in self.transformer_blocks:
            x_patches = transformer_block(x_patches)

        # Reshape back to 2D and classify
        x_transformed = x_patches.transpose(1, 2).view(B, -1, H, W)
        return self.classifier(x_transformed)


class HybridFPNTransformer(nn.Module):
    def __init__(self, num_classes, image_size=224, patch_size=16,
                 embed_dim=128, num_heads=8, transformer_depth=4):
        super().__init__()
        
        # Stem network remains the same - processes raw images
        self.stem = nn.Sequential(
            nn.Conv2d(3, embed_dim*2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim*2),
            nn.ReLU(inplace=True),
            nn.Conv2d(embed_dim*2, embed_dim, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(inplace=True)
        )

        # Patch embedding remains the same
        self.patch_embed = nn.Conv2d(
            embed_dim, embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )

        # Rotary positional embeddings for spatial awareness
        self.rotary_emb = RotaryPositionalEmbedding(embed_dim)

        # Transformer blocks remain unchanged
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, embed_dim)
            for _ in range(transformer_depth)
        ])

        # Simplified classification head - directly from transformer output
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(embed_dim, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # Initial feature extraction
        x = self.stem(x)
        x_patches = self.patch_embed(x)

        # Reshape for transformer processing
        B, C, H, W = x_patches.shape
        x_patches = x_patches.flatten(2).transpose(1, 2)

        # Add positional information
        rotary_pos_emb = self.rotary_emb(x_patches)
        x_patches = x_patches + rotary_pos_emb

        # Process through transformer blocks
        for transformer_block in self.transformer_blocks:
            x_patches = transformer_block(x_patches)

        # Reshape back to 2D and classify
        x_transformed = x_patches.transpose(1, 2).view(B, -1, H, W)
        return self.classifier(x_transformed)


In [6]:
def setup_logging(
    log_dir: str,
    log_level: int = logging.INFO,
    verbose: bool = False
) -> logging.Logger:
    os.makedirs(log_dir, exist_ok=True)

    # Generate more detailed log filename
    log_filename = os.path.join(
        log_dir,
        f'tumor_classification_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
    )

    # Configure logging with more details
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(name)s - %(levelname)s: %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger('TumorClassification')

    # Add hardware information logging
    if verbose:
        logger.info(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
        logger.info(f"CUDA Available: {torch.cuda.is_available()}")

    return logger

def calculate_metrics(true_labels, pred_labels, num_classes):
    # Calculate metrics with weighted average for multiclass
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')

    # Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)

    # Sensitivity (Per Class) derived from confusion matrix
    sensitivity = []
    for i in range(num_classes):
        tp = cm[i, i]  # True positives for class i
        fn = cm[i, :].sum() - tp  # False negatives for class i
        sensitivity.append(tp / (tp + fn) if (tp + fn) > 0 else 0)

    return {
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'sensitivity': sensitivity,
        'confusion_matrix': cm
    }
def save_metrics_to_csv(train_metrics, val_metrics, csv_path):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)

    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        headers = ['Epoch', 'Train_F1', 'Train_Precision', 'Train_Recall', 
                   'Train_Sensitivity', 'Train_Confusion_Matrix',
                   'Val_F1', 'Val_Precision', 'Val_Recall', 
                   'Val_Sensitivity', 'Val_Confusion_Matrix']
        writer.writerow(headers)
        
        # Write metrics for each epoch
        for epoch, (train, val) in enumerate(zip(train_metrics, val_metrics), start=1):
            writer.writerow([
                epoch,
                train['f1_score'],
                train['precision'],
                train['recall'],
                train['sensitivity'],
                str(train['confusion_matrix'].tolist()),  # Convert to string
                val['f1_score'],
                val['precision'],
                val['recall'],
                val['sensitivity'],
                str(val['confusion_matrix'].tolist())  # Convert to string
            ])
    print(f"Metrics saved to {csv_path}")

def plot_metrics(train_metrics, val_metrics, save_dir):
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Prepare metrics for plotting
    epochs = range(1, len(train_metrics) + 1)
    metrics_to_plot = ['f1_score', 'precision', 'recall', 'sensitivity']

    for metric in metrics_to_plot:
        plt.figure(figsize=(10, 6))
        train_values = [epoch_metrics[metric] if metric != 'sensitivity' else np.mean(epoch_metrics[metric]) for epoch_metrics in train_metrics]
        val_values = [epoch_metrics[metric] if metric != 'sensitivity' else np.mean(epoch_metrics[metric]) for epoch_metrics in val_metrics]

        plt.plot(epochs, train_values, label=f'Train {metric}', marker='o')
        plt.plot(epochs, val_values, label=f'Validation {metric}', marker='o')
        
        plt.title(f'{metric.capitalize()} over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)

        plt.ylim(0, 1)

        # Save the plot as an image
        save_path = os.path.join(save_dir, f'{metric}.png')
        plt.savefig(save_path)
        print(f"Saved plot for {metric} to {save_path}")
        plt.close()
        
def train_model_enhanced(
    model: nn.Module,
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    num_epochs: int = 150,
    device: Optional[torch.device] = None,
    num_classes: int = 4,
    log_dir: str = './logs',
    checkpoint_dir: str = '/working',
    model_name  = None,
    early_stopping_patience: int = 10,
    learning_rate_patience: int = 5
) -> Tuple[nn.Module, List[Dict], List[Dict]]:

    # Device management
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    checkpoint_dir = os.path.join(checkpoint_dir, model_name)
    log_dir = os.path.join(log_dir, model_name)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Logging and TensorBoard
    logger = setup_logging(log_dir, verbose=True)
    writer = SummaryWriter(log_dir=log_dir)

    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=learning_rate_patience
    )



    # Early stopping variables
    best_val_f1 = 0.0
    epochs_no_improve = 0

    train_metrics, val_metrics = [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_preds, train_true = [], []

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Optional gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_true.extend(labels.cpu().numpy())

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_preds, val_true = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_true.extend(labels.cpu().numpy())

        # Metrics calculation (similar to original function)
        train_metrics_epoch = calculate_metrics(
            np.array(train_true),
            np.array(train_preds),
            num_classes
        )
        val_metrics_epoch = calculate_metrics(
            np.array(val_true),
            np.array(val_preds),
            num_classes
        )

        # TensorBoard logging
        writer.add_scalar('Loss/Train', train_loss/len(train_loader), epoch)
        writer.add_scalar('Loss/Validation', val_loss/len(val_loader), epoch)
        writer.add_scalar('F1/Train', train_metrics_epoch['f1_score'], epoch)
        writer.add_scalar('F1/Validation', val_metrics_epoch['f1_score'], epoch)
        writer.add_scalar('Precision/Train', train_metrics_epoch['precision'], epoch)
        writer.add_scalar('Precision/Validation', val_metrics_epoch['precision'], epoch)
        writer.add_scalar('Recall/Train', train_metrics_epoch['recall'], epoch)
        writer.add_scalar('Recall/Validation', val_metrics_epoch['recall'], epoch)
        writer.add_scalar('Sensitivity/Train', np.mean(train_metrics_epoch['sensitivity']), epoch)
        writer.add_scalar('Sensitivity/Validation', np.mean(val_metrics_epoch['sensitivity']), epoch)

        # Learning rate scheduling
        scheduler.step(val_metrics_epoch['f1_score'])
        print(val_metrics_epoch['f1_score'])
        # Early stopping
        if val_metrics_epoch['f1_score'] > best_val_f1 and epoch >=10:
            best_val_f1 = val_metrics_epoch['f1_score']
            #print(best_val_f1)
            epochs_no_improve = 0

            # Save best model
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_f1': best_val_f1,
                'epoch': epoch
            }, os.path.join(checkpoint_dir, f'{model_name}_{epoch}_f1_{best_val_f1:.4f}.pth'))


        # Log epoch summary
        logger.info(f"Epoch {epoch+1}/{num_epochs}")
        logger.info(f"Train F1: {train_metrics_epoch['f1_score']:.4f}")
        logger.info(f"Val F1: {val_metrics_epoch['f1_score']:.4f}")

        # Early stopping check
        if epochs_no_improve >= early_stopping_patience:
            logger.info(f"Early stopping triggered after {epoch+1} epochs")
            break

        train_metrics.append(train_metrics_epoch)
        val_metrics.append(val_metrics_epoch)

    # Close TensorBoard writer
    writer.close()

    return model, train_metrics, val_metrics



In [7]:
import os
import re
import time
import torch

# Function to find highest F1 score files
def find_highest_f1_files():
    specific_folders = [
        {"folder": "VGG16", "loader": val_loader_base},
        {"folder": "AlexNet", "loader": val_loader_base},
        {"folder": "ResNet18", "loader": val_loader_base},
        {"folder": "HybridFPNTransformer", "loader": val_loader},
        {"folder": "HybridFPNTransformerGRY", "loader": val_loader_gry}
    ]

    highest_f1_files = []

    for item in specific_folders:
        folder_name = item["folder"]
        loader = item["loader"]
        folder_path = os.path.join("/working", folder_name)

        if os.path.isdir(folder_path):
            max_f1 = -1
            max_file = ""
            max_epoch = -1

            # Iterate through each file in the folder
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)

                # Use regex to extract the epoch and F1 score from the file name
                match = re.search(r"best_\w+_(\d+)_f1_(\d+\.\d+)\.pth", file_name)
                if match:
                    epoch = int(match.group(1))
                    f1_score = float(match.group(2))

                    # Update the highest F1 score and corresponding file path
                    if f1_score > max_f1 or (f1_score == max_f1 and epoch > max_epoch):
                        max_f1 = f1_score
                        max_file = file_path
                        max_epoch = epoch

            if max_file:
                highest_f1_files.append({"file": max_file, "loader": loader})

    return highest_f1_files

def evaluate_model(model_path, dataloader, device):
    # Determine the model architecture based on the file path
    if "VGG16" in model_path:
        model = models.vgg16(pretrained=False)
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)  # Adjust for 4 classes
    elif "ResNet18" in model_path:
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, 4)  # Adjust for 4 classes
    elif "AlexNet" in model_path:
        model = models.alexnet(pretrained=False)
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)  # Adjust for 4 classes
    elif "HybridFPNTransformerGRY" in model_path:
        model = HybridFPNTransformer_gry(num_classes=4)
    else:  # Default to HybridFPNTransformer
        model = HybridFPNTransformer(num_classes=4)

    # Load the model weights from the checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    model.to(device)  # Move model to the specified device
    model.eval()  # Set model to evaluation mode

    correct = 0
    total = 0
    inference_times = []

    # Iterate through the dataset
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Record start time
            start_time = time.time()

            # Perform inference
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # Record end time
            end_time = time.time()

            # Calculate inference time
            inference_times.append(end_time - start_time)

            # Update accuracy metrics
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average inference time and accuracy
    avg_inference_time = sum(inference_times) / len(inference_times)
    accuracy = 100 * correct / total

    return avg_inference_time, accuracy


def inference():
    # Default highest_f1_files
    highest_f1_files = [
        {'file': '/input/inference/pytorch/default/1/best_VGG16_51_f1_0.9977.pth', 'loader': val_loader_base},
        {'file': '/input/inference/pytorch/default/1/best_AlexNet_53_f1_0.9931.pth', 'loader': val_loader_base},
        {'file': '/input/inference/pytorch/default/1/best_ResNet18_50_f1_0.9970.pth', 'loader': val_loader_base},
        {'file': '/input/inference/pytorch/default/1/best_HybridFPNTransformer_54_f1_0.9809.pth', 'loader': val_loader},
        {'file': '/input/inference/pytorch/default/1/best_HybridFPNTransformerGRY_67_f1_0.9847.pth', 'loader': val_loader_gry}
    ]
    # highest_f1_files = find_highest_f1_files()
    print("highest_f1_files", highest_f1_files)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for item in highest_f1_files:
        model_path = item["file"]
        dataloader = item["loader"]
        avg_inference_time, accuracy = evaluate_model(model_path, dataloader, device)
        print(f"Model: {model_path}")
        print(f"Average Inference Time: {avg_inference_time:.4f} seconds")
        print(f"Accuracy: {accuracy:.2f}%")

In [8]:
EPOCHS_NUM = 160
# EPOCHS_NUM = 1
def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Log directory setup
    log_dir = './logs/'
    os.makedirs(log_dir, exist_ok=True)

    # # Define model dictionary
    # model_dict = {
    #     'VGG16': {
    #         'model': models.vgg16(),
    #         'weights_path': '/input/model/pytorch/default/1/vgg16-397923af.pth'
    #     },
    #     'ResNet18': {
    #         'model': models.resnet18(),
    #         'weights_path': '/input/model/pytorch/default/1/resnet18-f37072fd.pth'
    #     },
    #     'AlexNet': {
    #         'model': models.alexnet(),
    #         'weights_path': '/input/model/pytorch/default/1/alexnet-owt-7be5be79.pth'
    #     }
    # }

    # # CSV to store training time and parameters
    time_metrics = []

    # # Iterate through base models for experimentation
    # for model_name, model_info in model_dict.items():
    #     try:
    #         # Load pre-trained weights and adjust the classifier layer
    #         base_model = model_info['model']
    #         weights_path = model_info['weights_path']
    #         base_model.load_state_dict(torch.load(weights_path))
            
    #         if model_name in ['VGG16', 'AlexNet']:
    #             base_model.classifier[-1] = nn.Linear(base_model.classifier[-1].in_features, 4)  # Adjust for 4 classes
    #         elif model_name == 'ResNet18':
    #             base_model.fc = nn.Linear(base_model.fc.in_features, 4)  # Adjust for 4 classes
            
    #         base_model = base_model.to(device)

    #         # Define loss and optimizer
    #         criterion = nn.CrossEntropyLoss()
    #         optimizer = optim.Adam(
    #             base_model.parameters(),
    #             lr=1e-4,
    #             weight_decay=1e-5  # L2 regularization
    #         )

    #         # Record start time
    #         start_time = time.time()

    #         # Train the model
    #         trained_model, train_metrics, val_metrics = train_model_enhanced(
    #             model=base_model,
    #             train_loader=train_loader_base,  # Assuming this is defined elsewhere
    #             val_loader=val_loader_base,      # Assuming this is defined elsewhere
    #             criterion=criterion,
    #             optimizer=optimizer,
    #             num_epochs=EPOCHS_NUM,
    #             model_name=model_name,
    #             device=device,
    #             num_classes=4
    #         )

    #         # Record end time and calculate training duration
    #         end_time = time.time()
    #         training_time = end_time - start_time

    #         # Save metrics to CSV
    #         csv_path = os.path.join(log_dir, f'{model_name}_metrics.csv')
    #         save_metrics_to_csv(train_metrics, val_metrics, csv_path)

    #         # Save metric plots
    #         plot_path = os.path.join(log_dir, f'{model_name}_plots.png')
    #         plot_metrics(train_metrics, val_metrics, plot_path)

    #         # Log training time and parameters
    #         time_metrics.append({
    #             'Model': model_name,
    #             'Training Time (seconds)': training_time,
    #             'Number of Parameters': sum(p.numel() for p in base_model.parameters() if p.requires_grad)
    #         })

    #         # Clear VRAM
    #         del base_model, criterion, optimizer, trained_model
    #         torch.cuda.empty_cache()

    #     except Exception as e:
    #         print(f"Error processing {model_name}: {e}")

    # # # Save training time and parameter metrics to a CSV
    time_metrics_path = os.path.join(log_dir, 'training_time_metrics.csv')
    # pd.DataFrame(time_metrics).to_csv(time_metrics_path, index=False)

    # HybridFPNTransformer training
    hybrid_model = HybridFPNTransformer(num_classes=4).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        hybrid_model.parameters(),
        lr=1e-4,
        weight_decay=1e-5  # L2 regularization
    )

    start_time = time.time()
    trained_hybrid_model, hybrid_train_metrics, hybrid_val_metrics = train_model_enhanced(
        model=hybrid_model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=EPOCHS_NUM,
        device=device,
        model_name='Transformer',
        num_classes=4
    )
    end_time = time.time()
    hybrid_training_time = end_time - start_time

    csv_path = './logs/Transformer_metrics.csv'
    save_metrics_to_csv(hybrid_train_metrics, hybrid_val_metrics, csv_path)

    plot_path = './logs/Transformer_plots.png'
    plot_metrics(hybrid_train_metrics, hybrid_val_metrics, plot_path)

    # Log HybridFPNTransformer training time and parameters
    time_metrics.append({
        'Model': 'Transformer',
        'Training Time (seconds)': hybrid_training_time,
        'Number of Parameters': sum(p.numel() for p in hybrid_model.parameters() if p.requires_grad)
    })

    # Clear VRAM
    del hybrid_model, criterion, optimizer, trained_hybrid_model
    torch.cuda.empty_cache()

    pd.DataFrame(time_metrics).to_csv(time_metrics_path, index=False)

    hybrid_model_gry = HybridFPNTransformer_gry(num_classes=4).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        hybrid_model_gry.parameters(),
        lr=1e-4,
        weight_decay=1e-5  # L2 regularization
    )

    start_time = time.time()
    hybrid_model_gry, hybrid_train_metrics_gry, hybrid_val_metrics_gry = train_model_enhanced(
        model=hybrid_model_gry,
        # train_loader=train_dataset_gry,
        train_loader=train_loader_gry,
        val_loader=val_loader_gry,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=EPOCHS_NUM,
        device=device,
        model_name='HybridFPNTransformerGRY',
        num_classes=4
    )
    end_time = time.time()
    hybrid_gry_training_time = end_time - start_time

    csv_path = './logs/HybridFPNTransformerGRY_metrics.csv'
    save_metrics_to_csv(hybrid_train_metrics_gry, hybrid_val_metrics, csv_path)

    plot_path = './logs/HybridFPNTransformerGRY_plots.png'
    plot_metrics(hybrid_val_metrics_gry, hybrid_val_metrics, plot_path)

    # Log HybridFPNTransformer training time and parameters
    time_metrics.append({
        'Model': 'HybridFPNTransformerGRY',
        'Training Time (seconds)': hybrid_gry_training_time,
        'Number of Parameters': sum(p.numel() for p in hybrid_model_gry.parameters() if p.requires_grad)
    })

    # Clear VRAM
    # del hybrid_model_gry, criterion, optimizer, hybrid_model_gry
    del hybrid_model_gry, criterion, optimizer
    torch.cuda.empty_cache()

    pd.DataFrame(time_metrics).to_csv(time_metrics_path, index=False)

if __name__ == '__main__':
    main()
    # inference()

0.6434079745877987
0.6943618148260734
0.7229233605983775
0.7514865149849943
0.725989576520661
0.78371563767246
0.8223111660508293
0.8314123094583606
0.8411361745111943
0.851175348315338
0.8618600169824084
0.8969221000369552
0.883098443750513
0.8954407886028328
0.9019701007568945
0.9228700748866195
0.8922961451608985
0.9222732195239202
0.8682662149133128
0.8912527041363546
0.9371199223468786
0.943616063403138
0.9664044783418088
0.9611867522699907
0.9616767257144553
0.9423009083778476
0.9501262839924063
0.9010354533213821
0.9579420873653909
0.9700972298332052
0.9679993385923079
0.9724971666980311
0.9620803330326074
0.9708063826691773
0.9778655093902092
0.9677122460069181
0.9521909698193002
0.9582615803779487
0.9681131711269303
0.9695758389882516
0.9693405309936683
0.9739853188900299
0.9777933088844777
0.9685692331892316
0.9747465087930427
0.9754809716805165
0.9708403345517345
0.9754662899886485
0.9739420744738877
0.9747507776730139
0.9756053907586271
0.9754976986030446
0.9755690374044896